In [ ]:
GROQAPIKEY = "***"

In [20]:
import re
import fitz  # PyMuPDF
from pathlib import Path
from typing import List, Dict
from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers import SingleHopSpecificQuerySynthesizer
from ragas.testset.transforms import EmbeddingExtractor
from ragas import evaluate
from ragas import EvaluationDataset
from ragas.metrics import AnswerAccuracy, Faithfulness, ResponseRelevancy, AnswerCorrectness, ContextRelevance,  FactualCorrectness, ContextRecall
from ragas import SingleTurnSample
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from groq import Groq
from langchain_core.documents import Document
from langchain_community.llms import OpenAI
from typing import List, Dict
import re
import fitz
from pathlib import Path
import os
import requests
import json
import time

In [7]:
def clean_content(text: str) -> str:
    """Limpia el texto de una página según las reglas especificadas"""
    # 1. Eliminar encabezados de documento
    text = re.sub(r'-{3} DOCUMENTO: .*? -{3}', '', text, flags=re.DOTALL)
    
    # 2. Eliminar números de página y separadores
    text = re.sub(r'\n\d+\n', '\n', text)
    text = re.sub(r'─+', '', text)
    
    # 3. Eliminar repeticiones de títulos
    text = re.sub(r'Desarrollo de Software I\n+', '', text)
    
    # 4. Eliminar información personal y URLs
    text = re.sub(r'\b\w+\.\w+@\w+\.\w+\b', '', text)  # Emails
    text = re.sub(r'https?://\S+', '', text)  # URLs
    
    # 5. Unir líneas rotas y eliminar saltos innecesarios
    text = re.sub(r'-\n', '', text)  # Guiones de continuación
    text = re.sub(r'\n{3,}', '\n\n', text)  # Normalizar saltos
    text = re.sub(r'([a-záéíóúñ])\n([a-záéíóúñ])', r'\1 \2', text, flags=re.IGNORECASE) # Une líneas que terminan en letra (no punto) y comienzan con minúscula
    
    # 6. Eliminar contenido no relevante
    text = re.sub(r'Agenda\n.*?(?=\n\n)', '', text, flags=re.DOTALL)
    text = re.sub(r'Gracias\s*Desarrollo de Software I', '', text)
    
    # 7. Limpiar espacios en blanco
    text = '\n'.join([line.strip() for line in text.split('\n') if line.strip()])
    
    return text

def process_pdf_directory(input_dir: str) -> List[str]:
    """
    Procesa todos los PDFs en un directorio y devuelve una lista de chunks
    Cada chunk es el texto limpio de una página (string)
    """
    pdf_dir = Path(input_dir)    
    all_chunks = []
    
    for pdf_file in pdf_dir.glob('*.pdf'):
        try:
            with fitz.open(pdf_file) as doc:
                for page in doc:
                    raw_text = page.get_text()
                    cleaned_text = clean_content(raw_text)
                    
                    if cleaned_text.strip():
                        all_chunks.append(cleaned_text)
                        
    return all_chunks

if __name__ == "__main__":
    input_directory = "D:/Jupyter Notebooks/finetuning/pdfs"
    chunks = process_pdf_directory(input_directory)
    
    print(f"Chunks generados: {len(chunks)}")
    print("\nEjemplo de chunk:")
    print(chunks[:100])

Chunks generados: 325

Ejemplo de chunk:
['Mauricio   Gaona\n.co Profesor Facultad de Ingeniería Escuela de Ingeniería de Sistemas y Computación\n2025-I', 'Objetivos del curso Objetivo general Proporcionar al estudiante las bases conceptuales fundamentales de la ingeniería de software y los elementos\nmetodológicos necesarios para llevar a cabo el desarrollo de aplicaciones de software.\nObjetivos específicos Entender los conceptos fundamentales de la ingeniería de software.\nEntender y usar metodologías ágiles de desarrollo de software para ser un miembro\nefectivo y eficiente en un equipo de desarrollo ágil.\nEntender y aplicar los roles y las actividades que se realizan en las etapas de análisis,\ndiseño, codificación, pruebas y despliegue de una aplicación de software.\nEntender los conceptos principales de las metodologías tradicionales de desarrollo\nde software.', 'Objetivos del curso Objetivos específicos Entender como estimar un producto de software (tiempo y valor en $$$ de u

In [27]:
RETRY_DELAY = 5
def is_relevant(chunk: str, client) -> bool:
    print(f"\n[FILTRO RELEVANCIA] Analizando chunk ({len(chunk)} caracteres):")
    print("--------------------------------------------------")
    print(chunk[:200] + "..." if len(chunk) > 200 else chunk)

    SYS_PROMPT = """Evalúa si el texto contiene contenido académico o conceptual relevante para un curso de Desarrollo de Software
    (teorías, prácticas, metodologías, conceptos técnicos). Considera irrelevantes datos 
    personales, formatos vacíos o información administrativa.
    Responde solo con 'yes' o 'no'."""    
    while True:
        try:     
            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": SYS_PROMPT},
                    {"role": "user", "content": f"Texto:\n{chunk}"}                    
                ],
                model="llama3-70b-8192",
                temperature=0.1,
                max_tokens=100
            )
        
            decision = response.choices[0].message.content.lower().strip()
            print(f"\n→ Decisión de relevancia: {decision}")
            return decision.startswith('yes')
        except Exception as e:
            error_message = str(e)
            print(f"Error al procesar comprobar relevancia del chunk: {error_message}")

            if "Limit" in error_message and "TPD" in error_message:
                print("Se alcanzó el límite de tokens por día. Deteniendo el proceso.")
                return

            print(f"Reintentando en {RETRY_DELAY} segundos...")
            time.sleep(RETRY_DELAY)            



def generate_questions(chunk: str, client, max_questions=2):
    SYS_PROMPT = f"""Eres un generador de preguntas para un curso de Desarrollo de Software. Genera preguntas 
    específicas sobre el contenido academico presente en el texto. Reglas:
    1. Máximo {max_questions} preguntas
    2. Preguntas autónomas (no requieran contexto adicional)
    3. Cubrir los temas principales
    4. Usar solo información presente en el chunk
    5. Cada pregunta en una sola linea
    6. No hagas comentarios ni introducciones ni enumeraciones, solo pon cada pregunta en una sola linea."""

    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": SYS_PROMPT},
            {"role": "user", "content": f"Texto:\n{chunk}"}
        ],
        model="llama3-70b-8192",
        temperature=0.3,
        max_tokens=1000
    )
    output = response.choices[0].message.content.strip()
    questions = []    
    for line in output.split('\n'):
        line = line.strip()
        questions.append(line)
    return questions[:max_questions]

In [16]:
def load_state(filepath):
    """
    Carga el estado desde el archivo JSON.
    Si el archivo no existe, retorna un estado inicial.
    """
    if os.path.exists(filepath):
        with open(filepath, "r", encoding="utf-8") as file:
            return json.load(file)
    else:
        return {"currentChunkIdx": 0, "qaPairs": []}

def save_state(state, filepath):
    """
    Guarda el estado en el archivo JSON.
    """
    with open(filepath, "w", encoding="utf-8") as file:
        json.dump(state, file, ensure_ascii=False, indent=4)

In [28]:
STATE_FILE = r"D:\Jupyter Notebooks\Evaluation\generated_testset\generated_qapairs.json"

def generate_test_dataset(chunks: list[str], client) -> list[dict]:
    test_set = []
    print("\n" + "="*60)
    print(f"INICIANDO GENERACIÓN DE TEST SET ({len(chunks)} chunks)")
    print("="*60)
    state = load_state(STATE_FILE)
    start_idx = state["currentChunkIdx"] +1
    
    for idx, chunk in enumerate(chunks, 0):
        print(f"\n{'='*20} Procesando Chunk #{idx} {'='*20}")
        
        if not is_relevant(chunk, client):
            print(f"→ Chunk #{idx} descartado por irrelevante")
            continue
                    
        print(f"\nGenerando preguntas para el chunk #{idx}: {chunk}\n\n")
        while True:
            try:            
                questions = generate_questions(chunk, client)
                for question in questions:
                    expected_output = generate_expected_answer(question, chunk, client)
                    
                    state["qaPairs"].append({
                        "query": question,
                        "reference": chunk
                        
                    })
                    state["currentChunkIdx"] = idx
                save_state(state, STATE_FILE)                    
                break
            except Exception as e:
                error_message = str(e)
                print(f"Error al procesar chunk {idx}: {error_message}")

                if "Limit" in error_message and "TPD" in error_message:
                    print("Se alcanzó el límite de tokens por día. Deteniendo el proceso.")
                    return

                print(f"Reintentando en {RETRY_DELAY} segundos...")
                time.sleep(RETRY_DELAY)
    
    print("\n" + "="*60)
    print(f"PROCESO COMPLETADO")
    print("="*60)

In [30]:
generate_test_dataset(chunks, Groq(api_key=GROQAPIKEY))


INICIANDO GENERACIÓN DE TEST SET (325 chunks)

==================== Procesando Chunk #0 ====================

[FILTRO RELEVANCIA] Analizando chunk (107 caracteres):
--------------------------------------------------
Mauricio   Gaona
.co Profesor Facultad de Ingeniería Escuela de Ingeniería de Sistemas y Computación
2025-I

→ Decisión de relevancia: no
→ Chunk #0 descartado por irrelevante

==================== Procesando Chunk #1 ====================

[FILTRO RELEVANCIA] Analizando chunk (717 caracteres):
--------------------------------------------------
Objetivos del curso Objetivo general Proporcionar al estudiante las bases conceptuales fundamentales de la ingeniería de software y los elementos
metodológicos necesarios para llevar a cabo el desarro...

→ Decisión de relevancia: yes

Generando preguntas para el chunk #1: Objetivos del curso Objetivo general Proporcionar al estudiante las bases conceptuales fundamentales de la ingeniería de software y los elementos
metodológicos nec

In [19]:
RAG_URL = "http://127.0.0.1:8000/assistant/ask/"
def query_rag_api(question):
    """
    Realiza una petición POST al RAG y devuelve (answer, documents)
    """
    try:
        response = requests.post(
            RAG_URL,
            json={"question": question},
            headers={"Content-Type": "application/json"},
            timeout=60  
        )
        response.raise_for_status()
        
        data = response.json()
        return data.get("answer", ""), data.get("documents", [])
    
    except requests.exceptions.RequestException as e:
        print(f"Error al consultar RAG: {e}")
        return "", []

            
query_rag_api("Cuales son las caracteristicas de los requerimientos.")

('Según el Documento 4, las características de los requerimientos son:\n\n* No ambiguos: los requerimientos deben ser claros y no dejar espacio para interpretaciones múltiples.\n* Comprobables: los requerimientos deben ser verificables, es decir, debe ser posible comprobar si se cumplen o no.\n* Medibles: los requerimientos deben ser cuantificables, para poder evaluar si se han cumplido.\n* Necesarios: los requerimientos deben ser esenciales para la aceptación del producto o proceso.\n\nEn resumen, los requerimientos deben ser claros, verificables, cuantificables y esenciales para la aceptación del producto o proceso.',
 [{'id': None,
   'metadata': {'index': 39,
    'localIndex': 0,
    'maxLocalIndex': 0,
    'page': 30,
    'source': 'D:\\Universidad\\Noveno Semestre\\trabajo de grado\\profevardillabackend\\qaAssistant\\rag_pdf_data\\Clase1-DS1-2025-I-NP.pdf',
    'uniqueName': 'D:\\Universidad\\Noveno Semestre\\trabajo de grado\\profevardillabackend\\qaAssistant\\rag_pdf_data\\Clas

In [12]:
RETRY_DELAY = 5
def generate_expected_output(query, reference, client):
    while True:    
        try:        
            SYS_PROMPT = """Utiliza **solamente** el siguiente contexto para responder en español a la pregunta."""
            llm_response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": SYS_PROMPT},
                    {"role": "user", "content": f"Pregunta: {query}\n\nContexto:\n{reference}"}
                ],
                model="llama3-70b-8192",
                temperature=0.3,
                max_tokens=1000
            )
            return llm_response.choices[0].message.content.strip()
            
        except Exception as e:
            error_message = str(e)
            print(f"Error al generar expected output: {error_message}")

            if "Limit" in error_message and "TPD" in error_message:
                print("Se alcanzó el límite de tokens por día. Deteniendo el proceso.")
                return

            print(f"Reintentando en {RETRY_DELAY} segundos...")
            time.sleep(RETRY_DELAY)            
#res = generate_expected_output("¿Cuál es la principal diferencia entre los requerimientos funcionales y no funcionales en un sistema?",
#                               "Clasificación de los requerimientos.\nLos requerimientos funcionales\n•Declaraciones de los servicios que debe proporcionar el sistema, la forma en que el\nsistema debe reaccionar a las entradas y la forma en que el sistema debe comportarse\nen situaciones particulares.\nRequerimientos no funcionales\n•Limitaciones en los servicios o funciones ofrecidas por el sistema como de tiempo,\nlimitaciones en el proceso de desarrollo, de rendimiento, de calidad, normas,\ntecnología, etc",
#                       Groq(api_key=GROQAPIKEY))
#print(res)

In [21]:
with open("D:\Jupyter Notebooks\Evaluation\generated_testset\generated_testset.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
STATE_FILE = r"D:\Jupyter Notebooks\Evaluation\generated_testset\deepeval_dataset.json"
state = load_state(STATE_FILE)
start_idx = state["currentChunkIdx"] +1
client = Groq(api_key=GROQAPIKEY)
MAX_PAIRS = 250
for idx in range(start_idx, len(data['qaPairs'])):
    qa_pair = data['qaPairs'][idx]
    query = qa_pair['query']
    reference = qa_pair['reference']
    expected_response = generate_expected_output(query, reference, client)
    while True:
        try:
            answer, documents = query_rag_api(query)
            state["qaPairs"].append({
                "user_input": query,
                "retrieved_contexts": documents,
                "response": answer,
                "expected_response": expected_response,
                "reference": reference
            })
            state["currentChunkIdx"] = idx    
            save_state(state, STATE_FILE)
            break
        except Exception as e:
            error_message = str(e)
            print(f"Error al procesar chunk {idx}: {error_message}")
            
            if "Limit" in error_message and "TPD" in error_message:
                print("Se alcanzó el límite de tokens por día. Deteniendo el proceso.")
                exit()
            
            print(f"Reintentando en {RETRY_DELAY} segundos...")
            time.sleep(RETRY_DELAY)        

KeyboardInterrupt: 